# Verslag modelleren

*gemaakt door Kees Brouwer en Robbert van Nieuwkerk*

Eerst het importeren van de benodigde libraries

misschien handige code om te gebruiken:

%pwd (de data moet in je workdirectery staan)


In [1]:
import numpy as np
import pandas as pd
import datetime

er moet ook nog een keer worden uitgelegd wat er in welk df staat en wat er met ieder df is gedaan voordat het geimporteerd is. !TODO

In [2]:
dfProjectTasks = pd.read_excel('MODPRdataset.xlsx', sheet_name ='Project tasks')
dfAdditionalTasks = pd.read_excel('MODPRdataset.xlsx', sheet_name ='Additional tasks')
dfCrew = pd.read_excel('MODPRdataset.xlsx', sheet_name ='Crew')
dfRooms = pd.read_excel('MODPRdataset.xlsx', sheet_name ='Rooms')
dfHiring = pd.read_excel('MODPRdataset.xlsx', sheet_name ='Hiring')

Het schoonmaken van de cel, hierdoor kunnen we waardes makkelijker worden teruggevonden dan NaN

In [3]:
dfProjectTasks = dfProjectTasks.fillna(0)
dfAdditionalTasks = dfAdditionalTasks.fillna(0)
dfCrew = dfCrew.fillna(0)
dfRooms = dfRooms.fillna(1)

In [4]:
dfProjectTasks.replace("x", 1, inplace=True)
dfAdditionalTasks.replace("x", 1, inplace=True)
dfCrew.replace("x", 1, inplace=True)


**Belangrijke info voor het plannen**
- De ingehuurde mensen hebben skill 1 en 8 
- kort inhuren is maximaal 60 uur
- het bedrijf is van 9-18 open
- een crewlid kan niet meer dan 3 uur achter elkaar werken (hoe lang is de pauze die moet worden genomen? !TODO)
- een crewlid werkt nooit meer dan 7 uur op een dag
- kan een 'acquire skill' maar 1 keer per sprint worden gedaan of kunnen meerdere personen er een volgen? !TODO
- crewleden moet te samen aan de skills komen voor de opdracht.
- moeten de mensen die skill 1 nog niet hebben deze ook leren? !TODO
- kan een crewlid eerst een uur aan het ene project werken, dit niet afronden en dan nog aan een ander project? !TODO
- moet iedereen tegelijk aan het project werken

Stel we maken een soort kalender, startend op 1 januari 2020. 
Hiervan maken we een dataframe met de volgende kolommen:
- datum
- de taak
- persoon die het uitvoerd
- begintijd 
- eindtijd
- Tijdsduur?
- Of het crewlid is ingehuurd (T/F)
- Kosten zodra het crewlid is ingehuurd
- Of het een werkdag is (T/F)

In [5]:
data = pd.date_range("2020-01-01", periods=4*10*7, freq='D')

columns = ['Datum', 'Task','Crewlid', 'begintijd', 'eindtijd', 'tijdsduur', 'ingehuurd', 'kosten', 'werkdag']

dfKalender = pd.DataFrame(columns=columns)
dfKalender['Datum'] = data
dfKalender = dfKalender.fillna(0)
dfKalender.replace("0", False, inplace=True)


Het toevoegen of een dag een werkdag is. Er wordt in het weekend niet gewerkt dus deze dagen zullen afvallen.

In [6]:
dfKalender['werkdag'] = dfKalender['Datum'].apply(lambda Datum: Datum.isoweekday() in range (1,6))

Er moet een functie zijn die kan worden aangeroepen om te kijken of een taak door een bepaald persoon kan worden uitgevoerd.

Hier moeten de eisen van de opdracht worden meegegeven en een persoon. Vervolgens krijg je een boolean terug of het kan.

In [10]:
# een paar standaard variabele om de code leesbaar te maken.
rangeSkills = range(1,14)
rangeWorkdays = range(14,19)
rangeRooms = range(14,20)

functie waarbij de lijst van ReqSkill moet worden ingevuld van de Task en de skills van een persoon. De functie geeft een boolean terug of deze taak door deze persoon kan worden gedaan.

voorbeeld: <br>
OpdrachtUitvoeren(dfProjectTasks.iloc[0,rangeSkills],dfCrew.iloc[0,rangeSkills]) 
<br>uitkomst: False

In [12]:
def OpdrachtUitvoerenPersoonBool(eisen, persoon):
    for i in rangeSkills:
        result = (int(eisen[i-1]) <= 
                  int(persoon[i-1]))
        if result == False:
            break
    return result

de functie hieronder voegt de skills van personen samen en geeft de gezamelijke skills terug.

voorbeeld: PersonenSamenvoegen(dfCrew.iloc[0,rangeSkills],dfCrew.iloc[2,rangeSkills])
<br>
uitkomst: [1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1]

In [17]:
def PersonenSamenvoegen(persoon1, persoon2, persoon3 = None, persoon4 = None, persoon5 = None, persoon6 = None):
    combinedSkill = []
    for i in rangeSkills:
        combinedSkill.append(1 if persoon1[i-1]+persoon2[i-1] > 0 else 0) 
                             
    print (combinedSkill)